In [ ]:
import numpy as np
from copy import deepcopy
import time
import hashlib
from IPython.display import clear_output

In [ ]:
def read(filename):
    start = open(filename,'r').read().replace('\n', '').replace(' ', '')
    goal = open('goal.txt','r').read().replace('\n', '').replace(' ', '')
    start_state = []
    goal_state = []
    for i in range(0,len(start)):
        start_state.append(int(start[i]))
    for i in range(0,len(goal)):
        goal_state.append(int(goal[i]))
    return start_state, goal_state

In [ ]:
class State:

    def __init__(self, state, pre_move = 'root'):
        self.state = np.array([state[:3], state[3:6], state[6:]])
        self.pre_move = pre_move
        self.g = 0
        self.FindZero()

    def __eq__(self, other):
        if isinstance(other, State):
            return np.array_equal(self.state, other.state)
    
    def __hash__(self):
        return hash(tuple(map(tuple, self.state)))

    def FindZero(self):
        index = np.where(self.state == 0)
        # coor = list(zip(index[0], index[1]))
        try:
            coor = list(zip(index[0], index[1]))
        except:
           print(self.state)
           exit()
        return list(coor[0])

    def ValidMove(self, move):
        if move == 'up':
            if self.FindZero()[0] != 0:
                return True
        elif move == 'down':
            if self.FindZero()[0] != 2:
                return True
        elif move == 'left':
            if self.FindZero()[1] != 0:
                return True
        elif move == 'right':
            if self.FindZero()[1] != 2:
                return True
        else:
            print('invalid move')
            return False
    
    def SavedMove(self, move):
        if self.ValidMove(move):
            saved = deepcopy(self)
            row = self.FindZero()[0]
            col = self.FindZero()[1]
            if move == 'up':
                self.state[row][col] = self.state[row-1][col]
                self.state[row-1][col] = 0
                self.pre_move = 'up'
                self.g += 1
            elif move == 'down':
                self.state[row][col] = self.state[row+1][col]
                self.state[row+1][col] = 0
                self.pre_move = 'down'
                self.g += 1
            elif move == 'left':
                self.state[row][col] = self.state[row][col-1]
                self.state[row][col-1] = 0
                self.pre_move = 'left'
                self.g += 1
            elif move == 'right':
                self.state[row][col] = self.state[row][col+1]
                self.state[row][col+1] = 0
                self.pre_move = 'right'
                self.g += 1
            return saved
        else:
            return False
        
    def Expand(self):
        nodes = []
        action_list = ['up','down','left','right']
        loop_move = ['down','up','right','left']
        for action in range(0,4):
            if(self.ValidMove(action_list[action]) and self.pre_move != loop_move[action]):
                temp = deepcopy(self)
                temp.SavedMove(action_list[action])
                nodes.append(temp)
        return nodes

In [ ]:
def Visited(state, state_list):
    for s in state_list:
        if s == state:
            return True
    return False

def NumMisplaced(state, goal):
  if state.state.shape != goal.state.shape:
    print("Dimension Error")
    return False
  else:
    num_rows, num_cols = state.state.shape
    count = 0
    for i in range(0,num_rows):
      for j in range(0,num_cols):
        if state.state[i][j] != goal.state[i][j] and state.state[i][j] != 0:
          count += 1
    return count


def Aborted(state, goal):
  global n_expand, path, visitedNode
  global queue

  queue = dict(queue)
  # f = g + h
  # g = path length
  # h = number of current misplaced tiles
  '''
  1. Expand the node
  2. Calculate their respective f value
  3. Expand the nodes with the smallest f value with LIFO property
  4. Repeat untill solution found
  '''
  if Visited(state, visitedNode):
    return False

  visitedNode.append(state)

  if state == goal:
    return True
  
  else:
    for node in state.Expand():
      if not node:
        print('Expand fail')
      n_expand +=1
      # For each node append it into a queue (dictionary) and marking its f value
      queue[node] = node.g + NumMisplaced(node, goal)
    # Sort the queue according to f value
    queue = sorted(queue.items(), key=lambda x:x[1], reverse=False)
    # Expand nodes accord to the queue
    target_node = queue.pop(0)
    found = Aborted(target_node[0], goal)
  
    if found:
      print('found')
      return True

  visitedNode.remove(state)
  return False

In [ ]:
def alt(start,goal):

  global queue, n_expand
  free_set = [start]
  # visited_set = []

  while True:
    try:
      to_expand = free_set.pop(0)
    except:
      queue = dict(queue)
      queue = sorted(queue.items(), key=lambda x:x[1], reverse=False)
      target_node = queue.pop(0)
      to_expand= target_node[0]
    
    # if not Visited(to_expand, visited_set):
    #   # visited_set.append(to_expand)

    clear_output(wait=True)
    print(to_expand.state)
    print("Current g:", to_expand.g)
    print("Number of closed set:", len(visited_set))
    time.sleep(0.0000000000000000001)

    for node in to_expand.Expand():

      if node == goal:
        print('found')
        return True
      
      queue = dict(queue)
      queue[node] = node.g + NumMisplaced(node, goal)

    queue = sorted(queue.items(), key=lambda x:x[1], reverse=False)
    target_node = queue.pop(0)
    free_set.append(target_node[0])

In [ ]:
visitedNode = []
path = []
n_expand = 0
queue = {}

def main():
    start_state, goal_state = read('input.txt')
    test_state = [0,7,6,5,4,2,8,3,1]
    start = State(start_state)
    goal = State(goal_state)
    test = State(test_state)
    print(alt(start, goal))

main()